In [ ]:
import pandas as pd
import os
# FileFrame = pd.read_csv('/data/scratch/r098375/data/ACE/FileFrame.csv')
# Results = pd.read_csv('/data/scratch/r098375/data/ACE/post_processing/Predictions_DDS_model_epochs20_time_2023-11-19_22:56_ensamblePredictions.csv')
FileFrame = pd.read_csv('/home/soenke/Internship/DeepDicomSortFork/prototyping/ACE/FileFrame.csv')
Results = pd.read_csv('/home/soenke/Internship/DeepDicomSortFork/prototyping/ACE/Predictions_DDS_model_epochs20_time_2023-11-19_22:56_ensamblePredictions.csv')
# XNATFrame = pd.read_csv('/trinity/home/r098375/DDS/DeepDicomSort/.finalExperiments/exp4/scantype_mapping.csv', header=0, delimiter=';')

In [ ]:
Results_depr = Results[Results['slicenum']==10][['vote','NIFTI_path']]
Results_depr

In [ ]:
FileFrame['NIFTI_path'] = FileFrame['NIFTI_path'].apply(lambda x: str(x).replace('ACE_new','ACE'))

In [ ]:
CombinedFrame = Results_depr.merge(FileFrame, how='inner', left_on='NIFTI_path', right_on='NIFTI_path')
CombinedFrame = CombinedFrame.drop('sliced', axis=1)


In [ ]:
def unify_mapping_names(name):
    name = name.strip()
    return name.replace(' ', '_')

In [ ]:
def extract_scantype_name_from_datapath(datapath):
    datapath = datapath.split('/')[9]
    datapath = datapath.strip()
    datapath = '-'.join(datapath.split('-')[1:])
    return datapath

In [ ]:
CombinedFrame.head()

In [ ]:
CombinedFrame['extracted_scantype'] = CombinedFrame['originPath'].apply(extract_scantype_name_from_datapath)

In [ ]:
CombinedFrame[['originPath','extracted_scantype']]

In [ ]:
XNAT_Frame_cleanedNames = XNAT_Frame.apply(unify_mapping_names)

In [ ]:
def assign_label(extracted_scantype, xnat_frame_cleaned):
    cols = list(xnat_frame_cleaned.columns)
    for col in cols:
        if extracted_scantype in list(xnat_frame_cleaned.loc[:,col]):
            return col
    return None

In [ ]:
CombinedFrame['cleaned_label'] = CombinedFrame['extracted_scantype'].apply(assign_label, xnat_frame_cleaned=XNAT_Frame_cleanedNames)
CombinedFrame.to_csv('Annotated_File_Frame.csv', index=False)